# Universidade de Brasília
## Programa de Pós-Graduação em Computação Aplicada
## Mestrato Profissional em Computação Aplicada
### Mineração de Dados
#### Professor: Marcelo Ladeira
#### Alunos: 
##### 22000xxxx - Alisson Melo Rios
##### 22000xxxx - Filipe Oliveira da Silva
##### 220005559 - Luís Eduardo Barreiro de Jesus


## PREPARAÇÃO DO AMBIENTE DE PROCESSAMENTO

In [1]:
# Preparação do Ambiente
## Seção de bibliotecas
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
from mlxtend.frequent_patterns import apriori, association_rules

import json
import datetime 

import folium
from folium.plugins import HeatMap, HeatMapWithTime

from apyori import apriori

In [2]:
#import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, accuracy_score
from yellowbrick.classifier import ConfusionMatrix 

/Users/luisjesus/opt/anaconda3/lib/python3.9/site-packages/yellowbrick/style/colors.py:35: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  mpl_ge_150 = LooseVersion(mpl.__version__) >= "1.5.0"
/Users/luisjesus/opt/anaconda3/lib/python3.9/site-packages/setuptools/_distutils/version.py:351: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  other = LooseVersion(other)
/Users/luisjesus/opt/anaconda3/lib/python3.9/site-packages/yellowbrick/style/rcmod.py:31: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  mpl_ge_150 = LooseVersion(mpl.__version__) >= "1.5.0"
/Users/luisjesus/opt/anaconda3/lib/python3.9/site-packages/setuptools/_distutils/version.py:351: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  other = LooseVersion(other)


## MANIPULAÇÃO DE DADOS

Nesta seção, faz-se a manipulação dos dados de forma a se gerar as informações de acordo com a necessidade da análise dos dados.

### Colunas do Dataframe

Nesta subseção, trabalha-se a identificação de subgrupos de dados carregados de registros de Acidentes da PRF.

In [3]:
# Constantes
## Colunas do Dataframe e operações tipo 1
## sumarização da quantidade de pessoas e estado em consequência do acidente
col_oper1 = {'id':'count', 'pessoas':'sum', 'mortos':'sum', 'feridos_leves':'sum', 'feridos_graves':'sum', 'veiculos':'sum'}

## Colunas do Dataframe e operações tipo 2
col_oper2 = {'id':'count', 'mortos':'sum', 'feridos_leves':'sum', 'feridos_graves':'sum', 'veiculos':'sum'}

#------------------------------------------------------------
# Para a geração de subdatasets
#------------------------------------------------------------
## colunas dos registros de acidentes 
col_acidentes = ['id',  
                'data_inversa', 
                'dia_semana', 
                'horario', 
                'uf', 
                'br',
                'km',
                'municipio', 
                'causa_acidente', 
                'tipo_acidente',
                'classificacao_acidente', 
                'fase_dia', 
                'sentido_via',
                'condicao_metereologica', 
                'tipo_pista',
                'tracado_via', 
                'uso_solo',
                'latitude', 
                'longitude',
                'regional', 
                'delegacia', 
                'uop',
                'ilesos',
                'feridos_leves', 
                'feridos_graves', 
                'mortos',
                'ano_mes',
                'ano',
                'mes']

## colunas dos registros de veículos
col_veiculos = ['id',
                'id_veiculo', 
                'tipo_veiculo', 
                'marca', 
                'ano_fabricacao_veiculo',
                'ilesos',
                'feridos_leves', 
                'feridos_graves', 
                'mortos',
                'ano_mes',
                'ano',
                'mes']

## colunas dos registros de pessoas
col_pessoas =  ['id',
                'id_veiculo',
                'pesid',
                'tipo_envolvido', 
                'estado_fisico', 
                'idade', 
                'sexo']

### Carga do(s) arquivo(s) de registro de acidentes

A fonte de dados está disponível no sítio eletrônico de Dados Abertos da Polícia Rodoviária Federal, Órgão responsável, dentro outras, do patrulhamento das rodovias (CF 1988, art. 144, § 2º).

Link: https://www.gov.br/prf/pt-br/acesso-a-informacao/dados-abertos/dados-abertos-acidentes

In [25]:
# Carregar dados de acidentes - carregar de arquivos
df2021 = pd.read_csv("./dados/acidentes2021.csv", dtype={'br': 'str', 'id': 'int64', 'latitude':'float'}, encoding="Latin1", sep=";")

ValueError: could not convert string to float: '-24,56168114'

In [5]:
# Manipulação de campo data - a partir da coluna data_inversa
#df2021["ano-mes"] = df2021["data_inversa"].str.slice(start=0, stop=7)
# Transformar o campo no tipo Datetime
df2021["data_inversa"] = pd.to_datetime(df2021["data_inversa"])
# Criar campo ano
df2021["ano"] = df2021["data_inversa"].dt.year
# Criar campo mês
df2021["mes"] = df2021["data_inversa"].dt.month
# Campo com ano e mês
df2021["ano_mes"] = df2021['data_inversa'].dt.strftime('%B/%Y')
# Manipulando o campo br
df2021["br"] = "BR-" + df2021["br"].str.zfill(3)

# print no dataframe para conferência
df2021

,id,pesid,data_inversa,dia_semana,horario,uf,br,km,municipio,causa_acidente,...,feridos_graves,mortos,latitude,longitude,regional,delegacia,uop,ano,mes,ano_mes
0,331730,738258.0,2021-01-01,sexta-feira,05:30:00,SP,BR-116,453,REGISTRO,Reação tardia ou ineficiente do condutor,...,0,0,"-24,56168114","-47,86752174",SPRF-SP,DEL05-SP,UOP01-DEL05-SP,2021,1,January/2021
1,331804,738412.0,2021-01-01,sexta-feira,08:05:00,AM,BR-174,937,MANAUS,Reação tardia ou ineficiente do condutor,...,0,0,"-2,508068","-60,036434",SPRF-AM,DEL01-AM,UOP01-DEL01-AM,2021,1,January/2021
2,331815,738692.0,2021-01-01,sexta-feira,10:10:00,CE,BR-222,"128,5",ITAPAJE,Velocidade Incompatível,...,0,0,"-3,707626","-39,623509",SPRF-CE,DEL04-CE,UOP03-DEL04-CE,2021,1,January/2021
3,331815,738689.0,2021-01-01,sexta-feira,10:10:00,CE,BR-222,"128,5",ITAPAJE,Velocidade Incompatível,...,0,0,"-3,707626","-39,623509",SPRF-CE,DEL04-CE,UOP03-DEL04-CE,2021,1,January/2021
4,331823,740272.0,2021-01-01,sexta-feira,12:30:00,RJ,BR-493,18,MAGE,Ingestão de álcool e/ou substâncias psicoativa...,...,0,0,"-22,6687122","-43,01862696",SPRF-RJ,DEL02-RJ,UOP04-DEL02-RJ,2021,1,January/2021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150920,465965,1008243.0,2021-11-07,domingo,20:15:00,SP,BR-116,318,JUQUITIBA,Transitar no acostamento,...,1,0,"-23,92536222","-46,99376156",SPRF-SP,DEL04-SP,UOP01-DEL04-SP,2021,11,November/2021
150921,465993,1008346.0,2021-08-25,quarta-feira,15:10:00,RO,BR-364,970,PORTO VELHO,Acessar a via sem observar a presença dos outr...,...,0,0,"-9,65616271","-65,72389011",SPRF-RO,DEL01-RO,UOP03-DEL01-RO,2021,8,August/2021
150922,465993,1008343.0,2021-08-25,quarta-feira,15:10:00,RO,BR-364,970,PORTO VELHO,Acessar a via sem observar a presença dos outr...,...,1,0,"-9,65616271","-65,72389011",SPRF-RO,DEL01-RO,UOP03-DEL01-RO,2021,8,August/2021
150923,465993,1008344.0,2021-08-25,quarta-feira,15:10:00,RO,BR-364,970,PORTO VELHO,Acessar a via sem observar a presença dos outr...,...,1,0,"-9,65616271","-65,72389011",SPRF-RO,DEL01-RO,UOP03-DEL01-RO,2021,8,August/2021


In [6]:
# Montar a estrutura para montar o mapa com o histórico de acidentes
# >>> lista_tempo = []

#for x in df2021["mes"]:
#    mesinteiro = x
#    lista_tempo.append(datetime.date(1900, mesinteiro, 1).strftime('%B'))

#df2021["month_in_full"] = lista_tempo



# >>> dfbr = df2021["br"]
# >>> dfbr = dfbr.drop_duplicates(keep='last')
# >>> jsonbr = dfbr.to_json(orient="split")

#print(lista_rod)
#dfbr = df2021.iloc[:,["br"]]

#lista_br = []
#for i in lista_rod:
#    lista_br.append(i)

#print(lista_br)
#jsonbr = json.dumps(dfbr(_dict_))
                                      
#print(jsonbr)
#dfbr

#folium.LayerControl().add_to(jsonbr)

In [7]:
# criar geopandas
#br = gpd.read_file(r"PATH_FILE")
#br.plot()


In [8]:
# Seção de filtros - pegar somente os acidentes com vítimas - Fatais ou não
df2021 = df2021.loc[(df2021['br']=='BR-040')]

In [9]:
df2021

,id,pesid,data_inversa,dia_semana,horario,uf,br,km,municipio,causa_acidente,...,feridos_graves,mortos,latitude,longitude,regional,delegacia,uop,ano,mes,ano_mes
64,332159,740855.0,2021-01-02,sábado,22:45:00,RJ,BR-040,78,PETROPOLIS,Reação tardia ou ineficiente do condutor,...,0,0,"-22,48485874","-43,22843406",SPRF-RJ,DEL06-RJ,UOP02-DEL06-RJ,2021,1,January/2021
65,332159,740847.0,2021-01-02,sábado,22:45:00,RJ,BR-040,78,PETROPOLIS,Reação tardia ou ineficiente do condutor,...,0,0,"-22,48485874","-43,22843406",SPRF-RJ,DEL06-RJ,UOP02-DEL06-RJ,2021,1,January/2021
66,332159,740846.0,2021-01-02,sábado,22:45:00,RJ,BR-040,78,PETROPOLIS,Reação tardia ou ineficiente do condutor,...,0,0,"-22,48485874","-43,22843406",SPRF-RJ,DEL06-RJ,UOP02-DEL06-RJ,2021,1,January/2021
124,332508,740214.0,2021-01-04,segunda-feira,21:30:00,MG,BR-040,"820,8",SIMAO PEREIRA,Ausência de reação do condutor,...,0,0,"-21,95982293","-43,29917908",SPRF-MG,DEL05-MG,UOP01-DEL05-MG,2021,1,January/2021
125,332509,740152.0,2021-01-04,segunda-feira,21:15:00,RJ,BR-040,121,DUQUE DE CAXIAS,Ingestão de álcool pelo condutor,...,0,0,"-22,78185499","-43,28571273",SPRF-RJ,DEL01-RJ,UOP01-DEL01-RJ,2021,1,January/2021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150749,459999,993477.0,2021-12-06,segunda-feira,07:05:00,DF,BR-040,5,BRASILIA,Manobra de mudança de faixa,...,0,0,"-16,020903","-47,980541",SPRF-DF,DEL01-DF,UOP01-DEL01-DF,2021,12,December/2021
150750,459999,993476.0,2021-12-06,segunda-feira,07:05:00,DF,BR-040,5,BRASILIA,Manobra de mudança de faixa,...,0,0,"-16,020903","-47,980541",SPRF-DF,DEL01-DF,UOP01-DEL01-DF,2021,12,December/2021
150753,460189,994116.0,2021-05-14,sexta-feira,18:30:00,MG,BR-040,510,RIBEIRAO DAS NEVES,Condutor deixou de manter distância do veículo...,...,1,0,"-19,77463885","-44,12321882",SPRF-MG,DEL01-MG,UOP02-DEL01-MG,2021,5,May/2021
150754,460189,994115.0,2021-05-14,sexta-feira,18:30:00,MG,BR-040,510,RIBEIRAO DAS NEVES,Condutor deixou de manter distância do veículo...,...,0,1,"-19,77463885","-44,12321882",SPRF-MG,DEL01-MG,UOP02-DEL01-MG,2021,5,May/2021


### Dataframe dos Acidentes

In [10]:
# Criar Dataframe com os registros de acidentes (a estrutura do arquivo tem repetição nesses campos)
dfacid = df2021[col_acidentes]
# Total de registros antes de retirada das duplicações
len(dfacid)

7256

OBS:
- O total de registros do período é 150.919

In [11]:
# retirar as duplicações do acidentes
dfacid = dfacid.drop_duplicates(subset=['id'], keep='last')
# Total de registros únicos
len(dfacid)

3142

OBS:
- O total de acidentes do período é de 64.536;

### Estatística Descritiva dos dados de Acidentes

In [12]:
# imprimir os dados para verificação visual
dfacid.head(5)

,id,data_inversa,dia_semana,horario,uf,br,km,municipio,causa_acidente,tipo_acidente,...,regional,delegacia,uop,ilesos,feridos_leves,feridos_graves,mortos,ano_mes,ano,mes
66,332159,2021-01-02,sábado,22:45:00,RJ,BR-040,78,PETROPOLIS,Reação tardia ou ineficiente do condutor,Saída de leito carroçável,...,SPRF-RJ,DEL06-RJ,UOP02-DEL06-RJ,1,0,0,0,January/2021,2021,1
126,332509,2021-01-04,segunda-feira,21:15:00,RJ,BR-040,121,DUQUE DE CAXIAS,Ingestão de álcool pelo condutor,Colisão lateral,...,SPRF-RJ,DEL01-RJ,UOP01-DEL01-RJ,1,0,0,0,January/2021,2021,1
194,332741,2021-01-06,quarta-feira,09:10:00,MG,BR-040,"729,8",OLIVEIRA FORTES,Demais falhas mecânicas ou elétricas,Colisão com objeto,...,SPRF-MG,DEL05-MG,UOP01-DEL05-MG,1,0,0,0,January/2021,2021,1
272,332932,2021-01-07,quinta-feira,13:15:00,MG,BR-040,355,FELIXLANDIA,Demais falhas mecânicas ou elétricas,Saída de leito carroçável,...,SPRF-MG,DEL02-MG,UOP01-DEL02-MG,1,0,0,0,January/2021,2021,1
304,333060,2021-01-08,sexta-feira,07:10:00,MG,BR-040,716,BARBACENA,Velocidade Incompatível,Tombamento,...,SPRF-MG,DEL05-MG,UOP01-DEL05-MG,0,0,1,0,January/2021,2021,1


SUGESTÕES DE PERGUNTAS:
- Relação de causa do acidente x tipo do acidente x horario do acidente x condição meteorologica x tipo de pista x traçado da via x uso do solo x dia da semana com a mortalidade dos acidentes?
- Relação de tipos de veículos x mortalidade dos acidentes?
- Escolher um trecho de rodovia concedida para analisar. Exemplo: BR-040 DF até Juiz de Fora; 

#### Domínios do acidente

In [13]:
#lista = dfacid.causa_acidente.unique()
#lista = dfacid.tipo_acidente.unique()
#lista = dfacid.classificacao_acidente.unique()
#lista = dfacid.fase_dia.unique()
#lista = dfacid.sentido_via.unique()
#lista = dfacid.condicao_metereologica.unique()
#lista = dfacid.tipo_pista.unique()
#lista = dfacid.tracado_via.unique()
#lista = dfacid.uso_solo.unique()
#print(', '.join(lista))

Exemplo   | Valor do exemplo
--------- | ------
causa_acidente | Reação tardia ou ineficiente do condutor ,Velocidade Incompatível ,Demais falhas mecânicas ou elétricas ,Entrada inopinada do pedestre ,Condutor Dormindo ,Frear bruscamente ,Acessar a via sem observar a presença dos outros veículos ,Chuva ,Ausência de reação do condutor ,Pedestre andava na pista ,Problema com o freio ,Carga excessiva e/ou mal acondicionada ,Ingestão de álcool pelo condutor ,Acostamento em desnível ,Ultrapassagem Indevida ,Mal súbito do condutor ,Curva acentuada ,Avarias e/ou desgaste excessivo no pneu ,Acumulo de água sobre o pavimento ,Condutor deixou de manter distância do veículo da frente ,Pedestre cruzava a pista fora da faixa ,Falta de elemento de contenção que evite a saída do leito carroçável ,Manobra de mudança de faixa ,Demais falhas na via ,Condutor usando celular ,Pista Escorregadia ,Ingestão de álcool ou de substâncias psicoativas pelo pedestre ,Obstrução na via ,Ingestão de álcool e/ou substâncias psicoativas pelo pedestre ,Acumulo de óleo sobre o pavimento ,Ingestão de substâncias psicoativas pelo condutor ,Transitar na contramão ,Iluminação deficiente ,Pista esburacada ,Conversão proibida ,Afundamento ou ondulação no pavimento ,Animais na Pista ,Demais Fenômenos da natureza ,Transitar no acostamento ,Desrespeitar a preferência no cruzamento ,Falta de acostamento ,Objeto estático sobre o leito carroçável ,Pista em desnível ,Estacionar ou parar em local proibido ,Área urbana sem a presença de local apropriado para a travessia de pedestres ,Desvio temporário ,Declive acentuado ,Obras na pista ,Ausência de sinalização ,Acumulo de areia ou detritos sobre o pavimento ,Problema na suspensão ,Acesso irregular ,Retorno proibido ,Sinalização mal posicionada ,Deficiência do Sistema de Iluminação/Sinalização ,Faixas de trânsito com largura insuficiente ,Restrição de visibilidade em curvas horizontais ,Neblina ,Fumaça ,Faróis desregulados ,Sistema de drenagem ineficiente ,Restrição de visibilidade em curvas verticais ,Condutor desrespeitou a iluminação vermelha do semáforo ,Trafegar com motocicleta (ou similar) entre as faixas ,Redutor de velocidade em desacordo ,Deixar de acionar o farol da motocicleta (ou similar) ,Transitar na calçada ,Semáforo com defeito ,Participar de racha ,Sinalização encoberta ,Modificação proibida
tipo_acidente | Tombamento ,Incêndio ,Atropelamento de Pedestre ,Saída de leito carroçável ,Capotamento ,Colisão com objeto ,Colisão traseira ,Derramamento de carga ,Colisão lateral ,Eventos atípicos ,Colisão frontal ,Colisão lateral sentido oposto ,Colisão lateral mesmo sentido ,Atropelamento de Animal ,Colisão transversal ,Engavetamento ,Queda de ocupante de veículo
classificacao_acidente | Com Vítimas Feridas, Sem Vítimas, Com Vítimas Fatais
fase_dia | Amanhecer, Pleno dia, Plena Noite, Anoitecer
sentido_via | Crescente, Decrescente, Não Informado
condicao_metereologica | Nublado, Céu Claro, Chuva, Ignorado, Sol, Garoa/Chuvisco, Nevoeiro/Neblina, Vento, Neve, Granizo
tipo_pista | Dupla, Simples, Múltipla
tracado_via | Curva, Reta, Não Informado, Desvio Temporário, Retorno Regulamentado, Ponte, Interseção de vias, Rotatória, Túnel, Viaduto
uso_solo | Não, Sim



In [14]:
dictio = {'id':'count', 'ilesos':'sum','feridos_leves':'sum','feridos_graves':'sum', 'mortos':'sum'}

# Análise estatística dos dados de acidentes
def _acidentes(df, aggr_field, dicio ):
    ## Acidentes por ano_mes
    #dfagg_acid = dfacid.groupby(['causa_acidente']).agg({'id':'count', 'ilesos':'sum','feridos_leves':'sum','feridos_graves':'sum', 'mortos':'sum'})
    df = df.groupby(aggr_field).agg(dicio)
    # Altera o nome da coluna id para qtd - count(id)
    df.rename(columns={'id':'qtd'}, inplace=True)
    df['percentual_mortalidade'] = df['mortos'] / df['qtd'] * 100
    df.reset_index(inplace=True)
    df = df.sort_values(by=['percentual_mortalidade'],ascending=False)
    
    return df
# imprimir
#dfagg_acid


def _acidente_pessoas(df, aggr_field, dict_pess):
    print(aggr_field)
    df = df.groupby(aggr_field).agg(dict_pess)
    return df

In [15]:
aggr_fld = ['causa_acidente']

df1 = _acidentes(dfacid, aggr_fld, dictio)
df1

,causa_acidente,qtd,ilesos,feridos_leves,feridos_graves,mortos,percentual_mortalidade
55,Transitar na contramão,60,15,16,16,11,18.333333
16,Conversão proibida,53,15,21,8,6,11.320755
43,Pedestre andava na pista,36,16,5,7,4,11.111111
4,Acumulo de água sobre o pavimento,38,14,18,2,4,10.526316
13,Condutor Dormindo,134,29,70,21,11,8.208955
57,Ultrapassagem Indevida,38,14,11,8,3,7.894737
0,Acessar a via sem observar a presença dos outr...,217,118,58,21,13,5.990783
37,Mal súbito do condutor,37,3,22,10,2,5.405405
26,Entrada inopinada do pedestre,37,26,3,2,2,5.405405
54,Trafegar com motocicleta (ou similar) entre as...,79,30,34,8,4,5.063291


In [16]:
aggr_fld = ['tipo_acidente']
df1 = _acidentes(dfacid, aggr_fld, dictio)
df1

,tipo_acidente,qtd,ilesos,feridos_leves,feridos_graves,mortos,percentual_mortalidade
4,Colisão frontal,126,31,32,34,24,19.047619
12,Eventos atípicos,16,6,7,2,1,6.250000
1,Atropelamento de Pedestre,185,102,31,26,11,5.945946
7,Colisão lateral sentido oposto,36,14,14,5,2,5.555556
8,Colisão transversal,262,121,75,47,14,5.343511
16,Tombamento,259,46,153,47,10,3.861004
15,Saída de leito carroçável,518,114,290,67,16,3.088803
6,Colisão lateral mesmo sentido,327,167,113,29,10,3.058104
2,Capotamento,101,26,56,4,3,2.970297
9,Colisão traseira,636,317,213,67,17,2.672956


In [17]:
aggr_fld = ['classificacao_acidente']
df1 = _acidentes(dfacid, aggr_fld, dictio)
df1

,classificacao_acidente,qtd,ilesos,feridos_leves,feridos_graves,mortos,percentual_mortalidade
0,Com Vítimas Fatais,186,43,9,16,111,59.677419
1,Com Vítimas Feridas,2474,738,1300,394,0,0.000000
2,Sem Vítimas,482,399,0,0,0,0.000000


In [18]:
aggr_fld = ['fase_dia']
df1 = _acidentes(dfacid, aggr_fld, dictio)
df1

,fase_dia,qtd,ilesos,feridos_leves,feridos_graves,mortos,percentual_mortalidade
2,Plena Noite,1079,390,423,161,39,3.614458
3,Pleno dia,1769,665,781,213,63,3.561334
1,Anoitecer,155,70,53,18,5,3.225806
0,Amanhecer,139,55,52,18,4,2.877698


In [19]:
aggr_fld = ['tipo_pista']
df1 = _acidentes(dfacid, aggr_fld, dictio)
df1

,tipo_pista,qtd,ilesos,feridos_leves,feridos_graves,mortos,percentual_mortalidade
2,Simples,1032,335,455,153,52,5.038760
1,Múltipla,287,113,133,17,10,3.484321
0,Dupla,1823,732,721,240,49,2.687877


In [20]:
aggr_fld = ['tracado_via']
df1 = _acidentes(dfacid, aggr_fld, dictio)
df1

,tracado_via,qtd,ilesos,feridos_leves,feridos_graves,mortos,percentual_mortalidade
7,Rotatória,11,4,4,1,1,9.090909
2,Interseção de vias,71,33,22,9,6,8.450704
3,Não Informado,440,146,179,76,25,5.681818
0,Curva,642,197,321,83,24,3.738318
5,Reta,1823,732,726,218,53,2.907296
1,Desvio Temporário,79,30,29,14,2,2.531646
4,Ponte,8,2,4,2,0,0.000000
6,Retorno Regulamentado,47,20,21,5,0,0.000000
8,Túnel,1,1,0,0,0,0.000000
9,Viaduto,20,15,3,2,0,0.000000


In [21]:
aggr_fld = ['uso_solo']
df1 = _acidentes(dfacid, aggr_fld, dictio)
df1

,uso_solo,qtd,ilesos,feridos_leves,feridos_graves,mortos,percentual_mortalidade
0,Não,1641,522,761,223,75,4.570384
1,Sim,1501,658,548,187,36,2.398401


In [22]:
# Plotar o gráfico
#dfacid.plot(kind='bar', x='br', y='qtd')


In [23]:
# Análise estatística dos dados de acidentes
## Acidentes por UF
dfacid.describe()

,id,ilesos,feridos_leves,feridos_graves,mortos,ano,mes
count,3142.000000,3142.000000,3142.000000,3142.000000,3142.000000,3142.0,3142.000000
mean,369417.851369,0.375557,0.416614,0.130490,0.035328,2021.0,6.555060
std,21995.216612,0.484343,0.493076,0.336896,0.184636,0.0,3.400375
min,331709.000000,0.000000,0.000000,0.000000,0.000000,2021.0,1.000000
25%,350931.500000,0.000000,0.000000,0.000000,0.000000,2021.0,4.000000
50%,369306.000000,0.000000,0.000000,0.000000,0.000000,2021.0,7.000000
75%,387382.500000,1.000000,1.000000,0.000000,0.000000,2021.0,10.000000
max,460189.000000,1.000000,1.000000,1.000000,1.000000,2021.0,12.000000


### Plotar Mapa de Calor com tempo dos acidentes

In [24]:
###
# criar lista de ano_mes únicos
lista_index = df2021.ano_mes.unique()

print(lista_index)

lista_idx = []
for i in lista_index:
    lista_idx.append(i)

print(lista_idx)
# -------------------------------------------

# criar a weight list
weight_list = []
dfacid['conta'] = 1
for x in dfacid['ano_mes'].sort_values().unique():
    weight_list.append(dfacid.loc[dfacid['ano_mes'] == x, ["latitude", "longitude", "conta"]].groupby(["latitude", "longitude"]).sum().reset_index().values.tolist()) 

# -------------------------------------------

# criar o mapa base
my_map = folium.Map(
                location=(
                            (dfacid.latitude.mean()),
                            (dfacid.longitude.mean())
                        ),
                zoom_start=6
            )

#folium.LayerControl().add_to(my_map)
folium.TileLayer('stamentoner').add_to(my_map)

# criar o mapa de calor de acidentes
HeatMapWithTime(weight_list, radius=10, index=lista_idx,
                gradient={0.1: 'blue', 0.25: 'yellow', 0.75:'orange', 1:'red'},
                auto_play=True, min_opacity=0.5, max_opacity=1, use_local_extrema=True).add_to(my_map)

my_map

['January/2021' 'February/2021' 'March/2021' 'April/2021' 'May/2021'
 'June/2021' 'July/2021' 'August/2021' 'September/2021' 'October/2021'
 'November/2021' 'December/2021']
['January/2021', 'February/2021', 'March/2021', 'April/2021', 'May/2021', 'June/2021', 'July/2021', 'August/2021', 'September/2021', 'October/2021', 'November/2021', 'December/2021']


TypeError: Could not convert -22,48485874-22,78185499-21,32603098-18,72999125-21,25153502-20,34858326-17,67108001-21,38296999-22,11423526-22,378777-20,42042719-17,99610525-22,48005592-16,750555-18,12048563-17,70612195-22,4585308-19,28815-21,65915396-17,196358-20,52319889-19,20302941-17,70582651-19,28815-22,01143747-19,29012-22,67092706-18,220313-22,79965379-22,60246485-20,36303483-19,23100428-19,28815-22,22751025-18,51802789-21,447874-20,1565654-16,3181831-19,5511262-20,03136499-19,276977-22,7875769-21,71178893-22,556263-19,3946-22,18633705-22,56876595-20,99196931-22,56876595-17,96471403-22,00844904-16,713217-19,66067-21,4328731-21,31842199-19,35767518-18,48024596-19,92064782-22,06389311-22,571-22,58293864-22,10981577-22,20877957-21,447874-21,34218094-17,2454521-20,394899-19,61064-18,93592-16,847953-22,45304974-21,394039-19,90149502-16,925474-21,45724301-21,31842199-22,4585308-22,55763285-18,29623795-19,36133-19,7970544-22,65931496-21,98932602-20,0477638-21,520757-21,33378994-20,421508

In [ ]:
# criar o mapa base
my_map = folium.Map(
                location=[dfacid.latitude.mean(), dfacid.longitude.mean()]
                         ,tiles = "Stamen Terrain"
                         ,zoom_start = 9)
my_map

### Dataframe dos Veículos

In [ ]:
# Criar Dataframe com os veículos
dfveic = df2021[col_veiculos]
len(dfveic)

In [ ]:
dfveic

In [ ]:
# retirar as duplicações do veículos
dfveic = dfveic.drop_duplicates(subset=['id_veiculo'], keep='last')
# Total de registros únicos
len(dfveic)

In [ ]:
dfveic

In [ ]:
aggr_fld = ['tipo_veiculo']
df1 = _acidentes(dfveic, aggr_fld, dictio)
df1

### Dataframe das pessoas

In [ ]:
# Criar Dataframe com os veículos
dfpess = df2021[col_pessoas]
dfpess

In [ ]:
dfpess.describe()

In [ ]:
aggr_fld = ['tipo_envolvido', 'estado_fisico']
dict_pess={'tipo_envolvido':'count', 'idade':'mean'}
df = _acidente_pessoas(dfpess, aggr_fld, dict_pess)
df

## Regras de Associação - Apriori

Nesta seção vamos avaliar as regras de associação dos acidentes com o algoritmo Apriori.

### Preparação dos dados

Para execução do algoritmo, os dados devem ser transformados de DataFrame para Lista.

In [ ]:
# Fase 1... 
# Jogar os dados de acidentes em lista... somente campos a serem avaliados

# Vamos trabalhar com os seguintes campos: 
# 2   dia_semana
# 7   municipio
# 8   causa_acidente       
# 9   tipo_acidente      
# 10  classificacao_acidente       
# 11  fase_dia        
# 12  sentido_via      
# 13  condicao_metereologica       
# 14  tipo_pista       
# 15  tracado_via      
# 16  uso_solo     

col_apriori =  ['causa_acidente', 
                'tipo_acidente', 
                'fase_dia', 
                'condicao_metereologica', 
                'tipo_pista',
                'tracado_via',
                'classificacao_acidente']


df2apriori = dfacid[col_apriori]

acidentes = []
for i in range(0, len(df2apriori)):
#    print(i)
    acidentes.append([str(df2apriori.values[i,j]) for j in range(0,7)])
                     
#print(acidentes)

regras = apriori(acidentes, min_support = 0.3, min_confidence = 0.5, min_lift = 1, min_length=2)
#regras = apriori(acidentes)
resultados = list(regras)

resultados2 = [list(x) for x in resultados]
resultados2
print(len(resultados2))

resultadoFormatado = []
for j in range(0, 13):
    resultadoFormatado.append([list(x) for x in resultados2[j][2]])
    
resultadoFormatado


## Machine Learning - Classificação

In [ ]:
# Formato de matriz
previsores = df2apriori.iloc[:,0:6].values
classe = df2apriori.iloc[:,6].values
classe2 = df2apriori.iloc[:,6].values

In [ ]:
classe

In [ ]:
previsores

In [ ]:
# Transformação dos atributos categóricos em atributos numéricos, passando o índice de cada coluna categórica
# Precisamos criar um objeto para cada atributo categórico, pois na sequência vamos executar o processo de encoding novamente para o registro de teste
# Se forem utilizados objetos diferentes, o número atribuído a cada valor poderá ser diferente, o que deixará o teste inconsistente
labelencoder1 = LabelEncoder()
previsores[:,0] = labelencoder1.fit_transform(previsores[:,0])

labelencoder2 = LabelEncoder()
previsores[:,1] = labelencoder2.fit_transform(previsores[:,1])

labelencoder3 = LabelEncoder()
previsores[:, 2] = labelencoder3.fit_transform(previsores[:, 2])

labelencoder4 = LabelEncoder()
previsores[:, 3] = labelencoder4.fit_transform(previsores[:, 3])

labelencoder5 = LabelEncoder()
previsores[:, 4] = labelencoder5.fit_transform(previsores[:, 4])

labelencoder6 = LabelEncoder()
previsores[:, 5] = labelencoder6.fit_transform(previsores[:, 5])

#labelencoder7 = LabelEncoder()
#previsores[:, 6] = labelencoder7.fit_transform(previsores[:, 6])

#labelencoder8 = LabelEncoder()
#previsores[:, 7] = labelencoder8.fit_transform(previsores[:, 7])
labelencoder8 = LabelEncoder()
classe[:] = labelencoder8.fit_transform(classe[:])

In [ ]:
classe2

In [ ]:
# Divisão da base de dados entre treinamento e teste (30% para testar e 70% para treinar)
X_treinamento, X_teste, y_treinamento, y_teste = train_test_split(previsores,
                                                                  classe,
                                                                  test_size = 0.3,
                                                                  random_state = 0)
X_teste

In [ ]:
y_treinamento

In [ ]:
X_treinamento

In [ ]:
# Criação e treinamento do modelo (geração da tabela de probabilidades)
#naive_bayes = GaussianNB()
#naive_bayes.fit(X_treinamento, y_treinamento)

In [ ]:
# Previsões utilizando os registros de teste
#previsoes = naive_bayes.predict(X_teste)
#previsoes

In [ ]:
#geração da matriz de confusão e cálculo da taxa de acerto e erro
#confusao = confusion_matrix(y_teste, previsoes)
#confusao

In [ ]:
#taxa_acerto = accuracy_score(y_teste, previsoes)
#taxa_erro = 1 - taxa_acerto
#taxa_acerto

In [ ]:
# Visualização da matriz de confusão
# Warning interno da biblioteca yellowbrick, já esta na última versão (sem solução para o warning no momento)
v = ConfusionMatrix(GaussianNB())
v.fit(X_treinamento, y_treinamento)
v.score(X_teste, y_teste)
v.poof()

## k-means

In [ ]:
# Importação das bibliotecas
from sklearn import datasets
import numpy as np
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans

In [ ]:
# Carregamento da base de dados 
#iris = datasets.load_iris()
# visualização de quantos registros existem por classe
xx = df2apriori.to_numpy()

#iris
unicos, quantidade = np.unique(classe, return_counts = True)
unicos
#quantidade
#iris

In [ ]:
quantidade

In [ ]:
previsores

In [ ]:
cluster = KMeans(n_clusters = 3)
cluster.fit(previsores)

In [ ]:
# Visualização dos três centroides
centroides = cluster.cluster_centers_
centroides

In [ ]:
# Visualização dos grupos que cada registro foi associado
previsoes = cluster.labels_
previsoes

In [ ]:
# Contagem dos registros por classe
unicos2, quantidade2 = np.unique(previsoes, return_counts = True)
unicos2

In [ ]:
quantidade2

In [ ]:
classe

In [ ]:
previsoes

In [ ]:
# Geração da matriz de contingência para comparar os grupos com a base de dados
resultados = confusion_matrix(classe, previsoes)
#resultados

In [ ]:
# Geração do gráfico com os clusters gerados, considerando para um (previsoes 0, 1 ou 2)
# Usamos somente as colunas 0 e 1 da base de dados original para termos 2 dimensões
plt.scatter(previsores[previsoes == 0, 0], previsores[previsoes == 0, 1], 
            c = 'green', label = 'Com Vítimas Fatais')
plt.scatter(previsores[previsoes == 1, 0], previsores[previsoes == 1, 1], 
            c = 'red', label = 'Com Vítimas Feridas')
plt.scatter(previsores[previsoes == 2, 0], previsores[previsoes == 2, 1], 
            c = 'blue', label = 'Sem Vítimas')
plt.legend()

## Random Forest

In [ ]:
# Importação das bibliotecas
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.ensemble import RandomForestClassifier

In [ ]:
# Formato de matriz
previsores = df2apriori.iloc[:,0:6].values
classe = df2apriori.iloc[:,6].values
#classe2 = df2apriori.iloc[:,6].values


In [ ]:
previsores

In [ ]:
# Formato de matriz
previsores = df2apriori.iloc[:,0:6].values
classe = df2apriori.iloc[:,6].values
classe2 = df2apriori.iloc[:,6].values

In [ ]:
# Transformação dos atributos categóricos em atributos numéricos, passando o índice de cada atributo categórico
labelencoder = LabelEncoder()
previsores[:,0] = labelencoder.fit_transform(previsores[:,0])
previsores[:,1] = labelencoder.fit_transform(previsores[:,1])
previsores[:,2] = labelencoder.fit_transform(previsores[:,2])
previsores[:,3] = labelencoder.fit_transform(previsores[:,3])
previsores[:,4] = labelencoder.fit_transform(previsores[:,4])
previsores[:,5] = labelencoder.fit_transform(previsores[:,5])
#previsores[:,6] = labelencoder.fit_transform(previsores[:,6])
#previsores[:, 11] = labelencoder.fit_transform(previsores[:, 11])
#previsores[:, 13] = labelencoder.fit_transform(previsores[:, 13])
#previsores[:, 14] = labelencoder.fit_transform(previsores[:, 14])
#previsores[:, 16] = labelencoder.fit_transform(previsores[:, 16])
#previsores[:, 18] = labelencoder.fit_transform(previsores[:, 18])
#previsores[:, 19] = labelencoder.fit_transform(previsores[:, 19])

In [ ]:
# Divisão da base de dados entre treinamento e teste (30% para testar e 70% para treinar)
X_treinamento, X_teste, y_treinamento, y_teste = train_test_split(previsores,
                                                                  classe,
                                                                  test_size = 0.3,
                                                                  random_state = 0)

In [ ]:
X_treinamento

In [ ]:
y_treinamento

In [ ]:
# Criação do modelo, treinamento, obtenção das previsões e da taxa de acerto
floresta = RandomForestClassifier(n_estimators = 100)
floresta.fit(X_treinamento, y_treinamento)

In [ ]:
# Visualização dos atributos principais
floresta.estimators_
#floresta.estimators_[1]

In [ ]:
previsoes = floresta.predict(X_teste)
confusao = confusion_matrix(y_teste, previsoes)
taxa_acerto = accuracy_score(y_teste, previsoes)
taxa_acerto